In [15]:
import pandas as pd
import json
import numpy as np
from datetime import datetime

In [16]:
# load dataset products
f = open('products.json')
products = json.load(f)

f = open('orders.json')
orders = json.load(f)

In [17]:
products_df = pd.read_json("products.json")
products_df = products_df.drop(columns=["ProductCode", "ProductsBusinessLineLeader", "Subtype","BrandName", "BrandManager"])
products_df.head()

,BusinessUnit,Type,ProductName
0,LUXURY SPECIALTIES,ADVENTURING EQUIPMENT,Alchemy Jug (Blue)
1,COMMISSIONS,TOOLS & KITS,Legendary Crafting Components
2,LUXURY SPECIALTIES,MUSICAL INSTRUMENT,Siren Song Lyre
3,LUXURY SPECIALTIES,POTIONS & SCROLLS,Illuminators Tattoo Enchantment
4,ADVENTURING,ARMS & ARMOUR,Leather Armour


In [18]:
orders_df = pd.read_json("orders.json")
orders_df = orders_df.drop(columns=["OrderID", "CustomerID", "OrderDate", "DeliveryDate", "Territory", "productsIDs", "CartPriceInCP", "CartPrice"])
orders_df.head()

,Products,Quantities,ProductPricesInCP
0,"[Blood of the Lycanthrope Antidote, Blanket, R...","[8, 4, 9, 3]","[1128.0, 2728.0, 5517.0, 912.0]"
1,"[Sword of Life Stealing, Gloves of Swimming an...","[1, 4, 3]","[91.0, 868.0, 237.0]"
2,"[Crossbow bolts (100), Staff of Fire, Hazirawn...","[10, 2, 2, 2, 5]","[8370.0, 1966.0, 588.6, 1418.0, 1135.0]"
3,"[White Dragon Mask, Breastplate, Arrows (100),...","[9, 1, 7, 8, 3, 1, 1, 5]","[1908.0, 770.0, 2142.0, 4360.0, 1179.0, 587.0,..."
4,"[Horn, Rings of Shared Suffering, Signal whist...","[10, 3, 7]","[2940.0, 2268.0, 1974.0]"


In [19]:
products = [item for sublist in orders_df['Products'] for item in sublist]

quantities = [item for sublist in orders_df['Quantities'] for item in sublist]

prices = [item for sublist in orders_df['ProductPricesInCP'] for item in sublist]

expanded_df = pd.DataFrame({
    'Products': products,
    'Quantities': quantities,
    'ProductPricesInCP': prices
})
expanded_df = expanded_df.reset_index(drop=True)

In [20]:
expanded_df = pd.DataFrame({
    'Products': products,
    'Quantities': quantities,
    'ProductPricesInCP': prices
})

In [21]:
# Drop duplicates in the products_df DataFrame based on the ProductName column
products_df.drop_duplicates(subset=['ProductName'], keep='first', inplace=True)

# Remove trailing spaces from the ProductName column in products_df
products_df['ProductName'] = products_df['ProductName'].str.strip()

merged_df = pd.merge(expanded_df, products_df, how='left', left_on='Products', right_on='ProductName')
merged_df.drop(columns=['ProductName'], inplace=True)

merged_df.head()

final_df = merged_df.groupby(['BusinessUnit', 'Type']).agg({
    'Quantities': 'sum',
    'ProductPricesInCP': 'sum',
}).reset_index()


In [22]:

# Display the result
final_df.head()

,BusinessUnit,Type,Quantities,ProductPricesInCP
0,ADVENTURING,ADVENTURING EQUIPMENT,770505,378140209.0
1,ADVENTURING,ANIMALS & TRANSPORTATION,157648,84693670.2
2,ADVENTURING,ARMS & ARMOUR,951481,465134305.8
3,ADVENTURING,JEWELRY,93776,48203992.2
4,ADVENTURING,MUSICAL INSTRUMENT,115280,57311063.7


In [23]:
allTypes = final_df["Type"][:8]
totalQuantity = sum(final_df.Quantities)
categoryShare = np.zeros(len(allTypes))
categoryQuantities = np.zeros(len(allTypes))
for i, type in enumerate(allTypes):
    categoryQuantities[i] = sum (final_df[final_df.Type == type].Quantities)
    categoryShare[i] = categoryQuantities[i] / totalQuantity

In [24]:
category_df = pd.DataFrame({
    'Type' : allTypes, 
    'CategoryShare' : categoryShare,
    'CategoryQuantities' : categoryQuantities})
category_df

,Type,CategoryShare,CategoryQuantities
0,ADVENTURING EQUIPMENT,0.278483,1344504.0
1,ANIMALS & TRANSPORTATION,0.045506,219703.0
2,ARMS & ARMOUR,0.384520,1856446.0
3,JEWELRY,0.036693,177152.0
4,MUSICAL INSTRUMENT,0.028833,139203.0
5,POTIONS & SCROLLS,0.096061,463779.0
6,SUMMONING DEVICE,0.029229,141115.0
7,TOOLS & KITS,0.100675,486054.0


In [41]:
merge_df = (pd.merge(final_df, category_df) )
merge_df['BusinessUnitShare'] = merge_df['Quantities'] /merge_df['CategoryQuantities']
sorted_df = merge_df.sort_values(by = ['Type'], ascending=True)
sorted_df = sorted_df.sort_values(by = ['CategoryShare', 'BusinessUnit'], ascending=[False, True])


final1_df = sorted_df.reset_index(drop=True)

In [43]:
final1_df

,BusinessUnit,Type,Quantities,ProductPricesInCP,CategoryShare,CategoryQuantities,BusinessUnitShare
0,ADVENTURING,ARMS & ARMOUR,951481,465134305.8,0.384520,1856446.0,0.512528
1,COMMISSIONS,ARMS & ARMOUR,260317,130412395.3,0.384520,1856446.0,0.140223
2,LUXURY SPECIALTIES,ARMS & ARMOUR,644648,315011605.6,0.384520,1856446.0,0.347248
3,ADVENTURING,ADVENTURING EQUIPMENT,770505,378140209.0,0.278483,1344504.0,0.573078
4,COMMISSIONS,ADVENTURING EQUIPMENT,100876,55018009.7,0.278483,1344504.0,0.075028
5,LUXURY SPECIALTIES,ADVENTURING EQUIPMENT,473123,244977185.3,0.278483,1344504.0,0.351894
6,ADVENTURING,TOOLS & KITS,465898,229550205.5,0.100675,486054.0,0.958531
7,COMMISSIONS,TOOLS & KITS,3981,1452575.0,0.100675,486054.0,0.008190
8,LUXURY SPECIALTIES,TOOLS & KITS,16175,8894487.5,0.100675,486054.0,0.033278
9,ADVENTURING,POTIONS & SCROLLS,278944,130333248.0,0.096061,463779.0,0.601459


In [44]:
final1_df.to_csv('shopping.csv', index=False)  
final1_df.to_json('shopping.json')